# Dependencies

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import yfinance as yf

Fetch data

In [27]:
portfolio = {'NVDA': 0.5, 'SBUX': 0.5}
window_start = '2018-01-01'
window_end = '2023-01-01'
data_type = 'Adj Close'

data = yf.download(list(portfolio.keys()), start=window_start, end=window_end)[data_type]

[*********************100%%**********************]  2 of 2 completed


Process into returns (closing price -> percentage changes)

In [36]:
returns = data.pct_change().dropna()

returns

Ticker,NVDA,SBUX
Date,,
2018-01-03,0.065814,0.018740
2018-01-04,0.005271,0.003747
2018-01-05,0.008474,0.011539
2018-01-08,0.030641,-0.005033
2018-01-09,-0.000270,-0.002192
...,...,...
2022-12-23,-0.008671,0.005217
2022-12-27,-0.071353,0.009464
2022-12-28,-0.006019,-0.006048


Calculate expected returns. This time we will use historical average just to keep things simple. Other options include:
- CAPM, uses risk in calculation
- DMM or GGMN, better for dividend portfolios with predictable growth

In [37]:
expected_return = returns.mean()

expected_return

Ticker
NVDA    0.001401
SBUX    0.000702
dtype: float64

Convert series object into a pandas DataFrame for easier mapping in future (all the large set calculations should be done with already)

In [38]:
expected_return_df = expected_return.reset_index()
expected_return_df.columns = ['Ticker', 'Expected Return']

expected_return_df

,Ticker,Expected Return
0,NVDA,0.001401
1,SBUX,0.000702


Add weights to the portfolio dataframe (originally provided in the dictionary `portfolio`)

In [39]:
expected_return_df['Weight'] = expected_return_df['Ticker'].map(portfolio)

expected_return_df

,Ticker,Expected Return,Weight
0,NVDA,0.001401,0.5
1,SBUX,0.000702,0.5


Calculate the portfolio expected return

In [40]:
expected_return_df['Weighted Return'] = expected_return_df['Expected Return'] * expected_return_df['Weight']
portfolio_expected_return = expected_return_df['Weighted Return'].sum()

portfolio_expected_return

np.float64(0.0010515952685235428)